In [37]:
from utils import load_json
import pandas as pd
from collections import Counter

data = load_json("data\public_train_0316.jsonl")
data2 = load_json("data\public_train_0522.jsonl")

data = data + data2
data[:5]

claim = []
for item in data:
    item['claim'] = item['claim'].replace(" ", "")
    claim.append(item['claim'])

# data to pandas 
df = pd.DataFrame(data)
# use utf-8 to save chinese
# save data
df.to_json('data\merged.jsonl', orient='records', force_ascii=False, lines=True)


print(len(claim))
data = sorted(data, key=lambda x: x['label'] == 'NOT ENOUGH INFO')
print(len(set(claim)))


11620
11341


In [38]:
data_dict = {}
conflicting_claims = set()
for item in data:
    claim = item['claim']
    label = item['label']
    if label != 'NOT ENOUGH INFO':
        if claim in data_dict and data_dict[claim] != label:
            conflicting_claims.add(claim)
        else:
            data_dict[claim] = label

cleaned_data = [item for item in data if item['claim'] not in conflicting_claims]
claim = []
for item in cleaned_data:
    claim.append(item['claim'])
print(len(claim))
print(len(set(claim)))
print(conflicting_claims)


11596
11338
{'馬克思在《資本論》中論述了馬克思主義政治經濟學的實作。', '李祖德（TsuDerLee)是臺北出生的，他於臺北教育大學畢業，後從專業醫師轉身成爲經營者。', '木衛二十四伊俄卡斯忒名字由來爲希臘神話中俄狄浦斯的母親和妻子。'}


In [39]:

claim_counts = Counter(claim)
duplicate_claims = [item for item, count in claim_counts.items() if count > 1]

print(f'There are {len(duplicate_claims)} duplicate claims:')
for claim in duplicate_claims:
    print(claim)


There are 84 duplicate claims:
國立臺灣大學應用力學研究所從1984年開始招收碩、博士班研究生，首任所長爲理論及應用力學專家，美國國家工程院院士、中央研究院院士鮑亦興教授。
單片機的全稱是單晶片微處理器。
樂山大佛建造於唐朝而且花了90年。
中國人民解放軍武漢軍區，下轄湖北省軍區、河南省軍區。
歐洲民族中，以民族國家劃分各民族，同一國家中，使用相同語言的人群承認屬於同一族群，也有人更願意強調自己的國別而非語言。
高錳酸鉀的化學式是NaMnO4，是一個可溶於水、遇乙醇即被還原的強氧化劑。
李祖德（TsuDerLee)是臺北出生的，他於臺北醫學大學牙醫學系畢業，後從專業醫師轉身成爲經營者。
歐洲民族中各個民族基本是以民族國家劃分，同國家的使用同一語言的人群承認屬於同一族群，也有人更願意強調自己的國別而非語言。
李祖德（TsuDerLee)是北漂族，他於臺北醫學大學畢業，後從專業醫師轉身成爲經營者。
歐洲民族中各個族群基本是以民族國家劃分，同地區的使用同一語言的人群承認屬於同一族群，也有人更願意強調自己的國別。
粵菜在廣東貿易的興盛、外省移民與美國廚師的影響之下，料理越來越豐富。
單晶片微電腦也被稱作單片機。
樂山大佛修建了90年。
李祖德（TsuDerLee)是臺灣出生的，他於臺北醫學大學畢業，後從專業醫師轉身成爲經營者。
馬克思在《資本論》中論述了馬克思主義政治經濟學的進階概念。
李祖德（TsuDerLee)是在臺北市出生的，他於臺北醫學大學畢業，之後從專業醫師轉身成爲經營者。
宋朝宋高宗是北宋皇帝宋神宗的兒子，也是南宋第一任皇帝。
馬克思在自己的作品中論述了馬克思主義政治經濟學的基本概念。
廣東貿易的興盛、外省移民與南洋香料影響之下，粵菜料理越來越豐富。
馬克思在《資本論》中敘述了馬克思主義政治經濟學的基礎概念。
《春秋》三傳的其中一傳爲穀梁傳。
馬克思在《資本論：政治經濟學的批判》中論述了馬克思主義政治經濟學的基本概念。
歐洲民族中各個民族基本是以手指數量劃分，同國家的同一手指數量的人群承認屬於同一族群，也有人更願意強調自己的腳趾。
病理學是一門研究疾病的學科。
原產於馬來西亞和印尼等東南亞國家的榴槤雖然有濃重的氣味，仍因味道甜美而被廣泛食用，有「水果之王」的美譽。
南宋第一任皇帝宋高宗的父親是北宋皇帝宋徽宗。
病理學是研究疾病發生以

In [40]:
df[df['claim'] == "浙菜起源於中國浙江，整體口味偏清淡，北部偏甜，東部偏鹹，西部偏辣。"]

,id,label,claim,evidence
10631,17296,NOT ENOUGH INFO,浙菜起源於中國浙江，整體口味偏清淡，北部偏甜，東部偏鹹，西部偏辣。,"[[16766, None, None, None]]"
10925,17292,NOT ENOUGH INFO,浙菜起源於中國浙江，整體口味偏清淡，北部偏甜，東部偏鹹，西部偏辣。,"[[16802, None, None, None]]"


In [41]:
from collections import defaultdict

# Initialize a defaultdict with list
merged_data = defaultdict(list)

# Sort cleaned_data by 'claim'
cleaned_data = sorted(cleaned_data, key=lambda x: x['claim'])

# Iterate over the sorted data
for item in cleaned_data:
    id_ = item['id']
    claim = item['claim']
    evidence = item['evidence']
    label = item['label']

    # Check if claim is not in merged_data or if the label is not 'NOT ENOUGH INFO'
    if not merged_data[claim] or label != 'NOT ENOUGH INFO':
        if merged_data[claim]:
            # Add the new evidence to the existing evidence
            for e in evidence:
                for evi in e:
                    existing_evidences = [existing_e[2:] for sublist in merged_data[claim][0]['evidence'] for existing_e in sublist]
                    if evi[2:] not in existing_evidences:
                        merged_data[claim][0]['evidence'][0].append(evi)
            # Update id and label
            merged_data[claim][0]['id'] = max(merged_data[claim][0]['id'], id_)
            merged_data[claim][0]['label'] = label
        else:
            merged_data[claim].append({'id': id_, 'claim': claim, 'evidence': evidence, 'label': label})

# Convert back to a list of dictionaries
merged_data = [item for sublist in merged_data.values() for item in sublist]


In [42]:
# claim = "國立臺灣大學應用力學研究所從1984年開始招收碩、博士班研究生，首任所長爲理論及應用力學專家，美國國家工程院院士、中央研究院院士鮑亦興教授。"
for j in duplicate_claims:
    for i in merged_data:
        if i['claim'] == j:
            print(i)

{'id': 8501, 'claim': '國立臺灣大學應用力學研究所從1984年開始招收碩、博士班研究生，首任所長爲理論及應用力學專家，美國國家工程院院士、中央研究院院士鮑亦興教授。', 'evidence': [[[7667, 7249, '國立臺灣大學應用力學研究所', 3]]], 'label': 'supports'}
{'id': 1059, 'claim': '單片機的全稱是單晶片微處理器。', 'evidence': [[[512, 504, '單片機', 0], [512, 504, '微型計算機', 0]]], 'label': 'supports'}
{'id': 155, 'claim': '樂山大佛建造於唐朝而且花了90年。', 'evidence': [[[77, 48, '樂山大佛', 1], [10718, 9653, '樂山大佛', 0]]], 'label': 'supports'}
{'id': 363, 'claim': '中國人民解放軍武漢軍區，下轄湖北省軍區、河南省軍區。', 'evidence': [[[366, 375, '中國人民解放軍武漢軍區', 0]]], 'label': 'supports'}
{'id': 1257, 'claim': '歐洲民族中，以民族國家劃分各民族，同一國家中，使用相同語言的人群承認屬於同一族群，也有人更願意強調自己的國別而非語言。', 'evidence': [[[1064, 1107, '歐洲民族', 2], [1219, 1296, '歐洲民族', 1], [1088, 1137, '民族國家', 0], [1710, 1832, '歐洲民族', 0]]], 'label': 'supports'}
{'id': 4084, 'claim': '高錳酸鉀的化學式是NaMnO4，是一個可溶於水、遇乙醇即被還原的強氧化劑。', 'evidence': [[[3983, 4137, '高錳酸鉀', 0]]], 'label': 'refutes'}
{'id': 8274, 'claim': '李祖德（TsuDerLee)是臺北出生的，他於臺北醫學大學牙醫學系畢業，後從專業醫師轉身成爲經營者。', 'evidence': [[[7444, 7059, '李祖德', 0]]], 'la

In [43]:
claim = []
for item in merged_data:
    claim.append(item['claim'])
print(len(claim))

11338


In [44]:
# save data

# sort order columns label 
merged_data = sorted(merged_data, key=lambda x: x['label'] == 'NOT ENOUGH INFO')

merged_data = pd.DataFrame(merged_data)



merged_data.to_json('data\public_train.jsonl', orient='records', force_ascii=False, lines=True)

In [47]:

data = load_json("data\public_test_data.jsonl")
data2 = load_json("data\private_test_data.jsonl")
data = data+data2
for item in data:
    item['claim'] = item['claim'].replace(" ", "")

data = pd.DataFrame(data)
data.to_json('data\public_test.jsonl', orient='records', force_ascii=False, lines=True)

In [46]:
totalClaim = 0
missClaim = 0
missClaim_ = 0
cnt = 0
claim = ""
for i in range(len(merged_data)):
    totalClaim += 1
    claim = merged_data.loc[i]['claim']
    evidence = merged_data.loc[i]['evidence']
    label = merged_data.loc[i]['label']
    if label == 'NOT ENOUGH INFO':
        continue
    flag = False
    for e in evidence:
        for evi in e:
            if claim.find(evi[2:3][0])==-1:
                missClaim += 1
            else:
                flag = True
    if flag == False:
        missClaim_ += 1
print(totalClaim)
print(missClaim)
print(missClaim_)
print(missClaim/totalClaim)
print(missClaim_/totalClaim)

11338
2268
996
0.20003527959075676
0.08784618098430058
